# System Instructions: Crafting Effective System Prompts

## Introduction

In this notebook, you'll learn how to craft effective system prompts that define your agent's behavior, personality, and capabilities. System instructions are the foundation of your agent's context - they tell the LLM what it is, what it can do, and how it should behave.

### What You'll Learn

- What system instructions are and why they matter
- What belongs in system context vs. retrieved context
- How to structure effective system prompts
- How to set agent personality and constraints
- How different instructions affect agent behavior

### Prerequisites

- Completed Section 1 notebooks
- Redis 8 running locally
- OpenAI API key set

## Concepts: System Instructions

### What Are System Instructions?

System instructions (also called system prompts) are the **persistent context** that defines your agent's identity and behavior. They are included in every conversation turn and tell the LLM:

1. **Who it is** - Role and identity
2. **What it can do** - Capabilities and tools
3. **How it should behave** - Personality and constraints
4. **What it knows** - Domain knowledge and context

### System Context vs. Retrieved Context

| System Context | Retrieved Context |
|----------------|-------------------|
| **Static** - Same for every turn | **Dynamic** - Changes per query |
| **Role & behavior** | **Specific facts** |
| **Always included** | **Conditionally included** |
| **Examples:** Agent role, capabilities, guidelines | **Examples:** Course details, user preferences, memories |

### Why System Instructions Matter

Good system instructions:
- ✅ Keep the agent focused on its purpose
- ✅ Prevent unwanted behaviors
- ✅ Ensure consistent personality
- ✅ Guide tool usage
- ✅ Set user expectations

Poor system instructions:
- ❌ Lead to off-topic responses
- ❌ Cause inconsistent behavior
- ❌ Result in tool misuse
- ❌ Create confused or unhelpful agents

## Setup

In [2]:
# Environment setup
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("🔧 Environment Setup")
print("=" * 30)
print(f"OpenAI API Key: {'✅ Set' if OPENAI_API_KEY else '❌ Not set'}")

🔧 Environment Setup
OpenAI API Key: ✅ Set


In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

print("✅ Setup complete!")

✅ Setup complete!


## Hands-on: Building System Instructions.
Let's build system instructions for our Redis University Class Agent step by step.

### Example 1: Minimal System Instructions

Let's start with the bare minimum and see what happens.

In [4]:
# Minimal system prompt
minimal_prompt = "You are a helpful assistant."

# Test it
messages = [
    SystemMessage(content=minimal_prompt),
    HumanMessage(content="I need help planning my classes for next semester.")
]

response = llm.invoke(messages)
print("Response with minimal instructions:")
print(response.content)
print("\n" + "="*80 + "\n")

Response with minimal instructions:
Of course! I'd be happy to help you plan your classes. Could you please provide some more information to get started? Here are a few questions to consider:

1. **Major/Minor**: What is your major or field of study? Are there any specific requirements you need to fulfill?

2. **Current Progress**: How far along are you in your program (e.g., freshman, sophomore, junior, senior)? Are there any prerequisites or core courses you still need to complete?

3. **Interests**: Are there any particular subjects or courses you're interested in taking, even if they're outside your major?

4. **Schedule Preferences**: Do you prefer morning or afternoon classes? Are there any days you need to keep free for work or other commitments?

5. **Credits**: How many credits are you planning to take next semester?

6. **Advising**: Have you met with your academic advisor to discuss your degree plan and ensure you're on track?

Once you provide some more details, I can offer

**Problem:** The agent doesn't know it's a class scheduling agent. It might give generic advice instead of using our course catalog and tools.

### Example 2: Adding Role and Purpose

In [5]:
# Add role and purpose
role_prompt = """You are the Redis University Class Agent.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule
- Check prerequisites and eligibility
- Get personalized course recommendations
"""

# Test it
messages = [
    SystemMessage(content=role_prompt),
    HumanMessage(content="I need help planning my classes for next semester.")
]

response = llm.invoke(messages)
print("Response with role and purpose:")
print(response.content)
print("\n" + "="*80 + "\n")

Response with role and purpose:
Of course! I'd be happy to help you plan your classes for next semester. To start, could you provide me with some information about your current academic status and interests? Specifically:

1. What is your major or area of study?
2. Are there any specific courses you need to take next semester to meet graduation requirements?
3. Do you have any particular interests or topics you want to explore?
4. Are there any prerequisites or eligibility requirements you've already completed or need to check?
5. Do you prefer in-person, online, or hybrid classes?

Once I have this information, I can help you find courses that match your needs and preferences!




**Better!** The agent now understands its role, but it still doesn't know about our tools or how to behave.

### Example 3: Adding Behavioral Guidelines

In [6]:
# Add behavioral guidelines
behavior_prompt = """You are the Redis University Class Agent.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule
- Check prerequisites and eligibility
- Get personalized course recommendations

Guidelines:
- Be helpful, friendly, and encouraging
- Ask clarifying questions when needed
- Provide specific course recommendations with details
- Explain prerequisites and requirements clearly
- Stay focused on course planning and scheduling
- If asked about topics outside your domain, politely redirect to course planning
"""

# Test with an off-topic question
messages = [
    SystemMessage(content=behavior_prompt),
    HumanMessage(content="What's the weather like today?")
]

response = llm.invoke(messages)
print("Response to off-topic question:")
print(response.content)
print("\n" + "="*80 + "\n")

Response to off-topic question:
I'm here to help you with course planning and academic scheduling. If you're interested in finding courses that match your interests or need help planning your schedule, feel free to ask! For weather information, I recommend checking a reliable weather website or app.




**Great!** The agent now stays focused on its purpose and redirects off-topic questions.

### Example 4: Complete System Instructions

Let's build the complete system instructions for our agent.

In [7]:
# Complete system instructions
complete_prompt = """You are the Redis University Class Agent, powered by Redis and the Agent Memory Server.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule for upcoming semesters
- Check prerequisites and course eligibility
- Get personalized course recommendations based on their goals

You have access to:
- A complete course catalog with descriptions, prerequisites, and schedules
- Student preferences and goals (stored in long-term memory)
- Conversation history (stored in working memory)
- Tools to search courses and check prerequisites

Guidelines:
- Be helpful, friendly, and encouraging
- Ask clarifying questions when you need more information
- Provide specific course recommendations with course codes and details
- Explain prerequisites and requirements clearly
- Remember student preferences and reference them in future conversations
- Stay focused on course planning and scheduling
- If asked about topics outside your domain, politely redirect to course planning

Example interactions:
- Student: "I'm interested in machine learning"
  You: "Great! I can help you find ML courses. What's your current year and have you taken any programming courses?"

- Student: "What are the prerequisites for CS401?"
  You: "Let me check that for you." [Use check_prerequisites tool]
"""

print("Complete system instructions:")
print(complete_prompt)

Complete system instructions:
You are the Redis University Class Agent, powered by Redis and the Agent Memory Server.

Your role is to help students:
- Find courses that match their interests and requirements
- Plan their academic schedule for upcoming semesters
- Check prerequisites and course eligibility
- Get personalized course recommendations based on their goals

You have access to:
- A complete course catalog with descriptions, prerequisites, and schedules
- Student preferences and goals (stored in long-term memory)
- Conversation history (stored in working memory)
- Tools to search courses and check prerequisites

Guidelines:
- Be helpful, friendly, and encouraging
- Ask clarifying questions when you need more information
- Provide specific course recommendations with course codes and details
- Explain prerequisites and requirements clearly
- Remember student preferences and reference them in future conversations
- Stay focused on course planning and scheduling
- If asked about

## Testing: Compare Different Instructions

Let's test how different system instructions affect agent behavior.

In [8]:
# Test query
test_query = "I want to learn about databases but I'm not sure where to start."

# Test with different prompts
prompts = {
    "Minimal": minimal_prompt,
    "With Role": role_prompt,
    "With Behavior": behavior_prompt,
    "Complete": complete_prompt
}

for name, prompt in prompts.items():
    messages = [
        SystemMessage(content=prompt),
        HumanMessage(content=test_query)
    ]
    response = llm.invoke(messages)
    print(f"\n{'='*80}")
    print(f"{name} Instructions:")
    print(f"{'='*80}")
    print(response.content)
    print()


Minimal Instructions:
Learning about databases is a great way to enhance your technical skills, as they are fundamental to storing, retrieving, and managing data in many applications. Here’s a structured approach to get you started:

1. **Understand the Basics:**
   - **What is a Database?** Learn about what databases are, their purpose, and how they are used in applications.
   - **Types of Databases:** Understand the difference between relational databases (SQL) and non-relational databases (NoSQL).

2. **Relational Databases (SQL):**
   - **Key Concepts:** Familiarize yourself with tables, rows, columns, primary keys, foreign keys, and relationships.
   - **SQL Language:** Learn the basics of SQL (Structured Query Language), which is used to interact with relational databases.
     - **Basic SQL Queries:** SELECT, INSERT, UPDATE, DELETE.
     - **Advanced SQL:** Joins, subqueries, indexes, and transactions.
   - **Popular RDBMS:** Explore some popular relational database management

## Key Takeaways

### What to Include in System Instructions

1. **Identity & Role**
   - Who the agent is
   - What domain it operates in

2. **Capabilities**
   - What the agent can do
   - What tools/data it has access to

3. **Behavioral Guidelines**
   - How to interact with users
   - When to ask questions
   - How to handle edge cases

4. **Constraints**
   - What the agent should NOT do
   - How to handle out-of-scope requests

5. **Examples** (optional)
   - Sample interactions
   - Expected behavior patterns

### Best Practices

✅ **Do:**
- Be specific about the agent's role
- Include clear behavioral guidelines
- Set boundaries for out-of-scope requests
- Use examples to clarify expected behavior
- Keep instructions concise but complete

❌ **Don't:**
- Include dynamic data (use retrieved context instead)
- Make instructions too long (wastes tokens)
- Be vague about capabilities
- Forget to set constraints
- Include contradictory guidelines

## Exercises

1. **Modify the system instructions** to make the agent more formal and academic in tone. Test it with a few queries.

2. **Add a constraint** that the agent should always ask about the student's year (freshman, sophomore, etc.) before recommending courses. Test if it follows this constraint.

3. **Create system instructions** for a different type of agent (e.g., a library assistant, a gym trainer, a recipe recommender). What changes?

4. **Test edge cases**: Try to make the agent break its guidelines. What happens? How can you improve the instructions?

## Summary

In this notebook, you learned:

- ✅ System instructions define your agent's identity, capabilities, and behavior
- ✅ System context is static (same every turn) vs. retrieved context is dynamic
- ✅ Good instructions include: role, capabilities, guidelines, constraints, and examples
- ✅ Instructions significantly affect agent behavior and consistency
- ✅ Start simple and iterate based on testing

**Next:** In the next notebook, we'll define tools that give our agent actual capabilities to search courses and check prerequisites.